# Ollama Model Manager

This notebook allows you to manage LLM models on your Ollama deployment through a simple FIT command. Specify task=pull to download and task=delete to delete models.

Example usage:
| makeresults | fit MLTKContainer algo=llm_rag_ollama_model_manager task=pull model_name=mistral _time into app:llm_rag_ollama_model_manager

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import requests
import os
from sentence_transformers import SentenceTransformer
# ...
# global constants
ollama_url = "http://ollama:11434"
MODEL_DIRECTORY = "/srv/app/model/data/"

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [2]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("llm_rag_ollama_model_manager")
print(df.describe())
print(param)

              _time
count  1.000000e+00
mean   1.715575e+09
std             NaN
min    1.715575e+09
25%    1.715575e+09
50%    1.715575e+09
75%    1.715575e+09
max    1.715575e+09
{'options': {'params': {'mode': 'stage', 'algo': 'ollama_model_manager', 'manager': 'pull', 'model': 'mistral'}, 'args': ['_time'], 'feature_variables': ['_time'], 'model_name': 'ollama_model_manager', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '60000', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['_time']}


## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
def apply(model,df,param):
    manager = param['options']['params']['task'].strip('\"')
    try:
        model_type = param['options']['params']['model_type'].strip('\"')
    except:
        model_type = "LLM"
    if model_type == "embedder_model":
        # embedder model
        if manager == "pull":
            model_name = param['options']['params']['model_name'].strip('\"').strip('\'')
            modelPath = os.path.join("/srv/app/model/data", model_name)
            try:
                os.makedirs(modelPath)
            except:
                print("path already exists")
            model = SentenceTransformer(model_name)
            model.save(modelPath)
            l = model_name
            m = f"Downloaded embedder model {model_name}"
        else:
            print("Not supported")
            l = "None"
            m = "Not supported task"
    else:
        if manager == "pull":
            # Download specified model
            try:
                model_name = param['options']['params']['model_name'].strip('\"')
                uri = f"{ollama_url}/api/pull"
                data = {
                    "name": model_name
                }
                data = json.dumps(data)
                requests.post(uri, data=data)
                m = f'Pulled model {model_name}.'
            except:
                m = f'ERROR during model download.'
            
        elif manager == "delete":
            # Delete specified model
            model_name = param['options']['params']['model_name'].strip('\"')
            uri = f"{ollama_url}/api/delete"
            data = {
                "name": model_name
            }
            data = json.dumps(data)
            requests.delete(uri, data=data)
            m = f'Deleted model {model_name}.'
        else:
            m = "No task specified"
        
        # List all existing models    
        uri = f"{ollama_url}/api/tags"
        response = requests.get(uri).json()
        response = response['models']
        try:
            l = ""
            for r in response:
                l += r['model'].split(":")[0]
                l += " "
        except:
            l = None
    l = [l]
    m = [m]
    cols={'Models': l, 'Message': m}
    returns=pd.DataFrame(data=cols)
    return returns

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

def compute(model,df,param):
    manager = param['params']['task'].strip('\"')
    try:
        model_type = param['params']['model_type'].strip('\"')
    except:
        model_type = "LLM"
    
    if model_type == "embedder_model":
        # embedder model
        if manager == "pull":
            model_name = param['params']['model_name'].strip('\"').strip('\'')
            modelPath = os.path.join("/srv/app/model/data", model_name)
            try:
                os.makedirs(modelPath)
            except:
                print("path already exists")
            model = SentenceTransformer(model_name)
            model.save(modelPath)
            l = model_name
            m = f"Downloaded embedder model {model_name}"
        else:
            print("Not supported")
            l = "None"
            m = "Not supported task"
                  
    else:
        # LLM model
        if manager == "pull":
            # Download specified model
            try:
                model_name = param['params']['model_name'].strip('\"')
                uri = f"{ollama_url}/api/pull"
                data = {
                    "name": model_name
                }
                data = json.dumps(data)
                requests.post(uri, data=data)
                m = f'Pulled model {model_name}.'
            except:
                m = f'ERROR during model download.'
            
        elif manager == "delete":
            # Delete specified model
            model_name = param['params']['model_name'].strip('\"')
            uri = f"{ollama_url}/api/delete"
            data = {
                "name": model_name
            }
            data = json.dumps(data)
            requests.delete(uri, data=data)
            m = f'Deleted model {model_name}.'
        else:
            m = "No task specified"
        
        # List all existing models    
        uri = f"{ollama_url}/api/tags"
        response = requests.get(uri).json()
        response = response['models']
        try:
            l = ""
            for r in response:
                l += r['model'].split(":")[0]
                l += " "
        except:
            l = None

    cols={'Models': l, 'Message': m}
    returns= [cols]
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code